In [ ]:
import pandas as pd
from phonenumbers.phonenumberutil import number_type
 

import pandas as pd

# Загрузка данных
df = pd.read_csv('dirty.csv')

# Убедимся, что даты в формате datetime
df['create_date'] = pd.to_datetime(df['create_date'], errors='coerce')
df['update_date'] = pd.to_datetime(df['update_date'], errors='coerce')

# Функция для создания золотой записи с учетом актуальности данных
def create_gold_record(group):
    # Если есть update_date, сортируем группу по нему в убывающем порядке
    sorted_group = group.sort_values(by=['update_date', 'create_date'], ascending=False)
    
    # Начинаем с первой строки (самой актуальной по дате)
    gold_record = sorted_group.iloc[0].copy()
    
    # Заполняем пропуски данными из остальных строк в порядке приоритета
    for column in group.columns:
        if pd.isna(gold_record[column]):  # Если значение пустое
            non_null_values = sorted_group[column].dropna()
            if not non_null_values.empty:
                gold_record[column] = non_null_values.iloc[0]  # Берем первое непустое значение
    
    return pd.Series(gold_record)

# Группировка по client_inn и создание золотых записей
golden_records = df.groupby('client_inn').apply(create_gold_record)

# Сброс индекса и сохранение результата
golden_records = golden_records.reset_index(drop=True)
golden_records.to_csv('golden_records.csv', index=False)

print("Золотые записи с учетом актуальности сохранены в 'golden_records.csv'")

print("Данные сохранены в файл output_zips.txt")

C:\Users\aangm\AppData\Local\Temp\ipykernel_13968\1439958989.py:14: DtypeWarning: Columns (18,19,20,25,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dirty.csv')
C:\Users\aangm\AppData\Local\Temp\ipykernel_13968\1439958989.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  golden_records = df.groupby('client_inn').apply(create_gold_record)


Золотые записи с учетом актуальности сохранены в 'golden_records.csv'
Данные сохранены в файл output_zips.txt
